### Подготовка данных
Для формирования обучающей выборки из датасета с длинной историей выбран подход, при котором в нее с большей вероятностью попадают недавние заказы.

In [ ]:
start_date = pd.to_datetime(orders.created_at.min())
end_date = pd.to_datetime(orders.created_at.max())
day_delta = (end_date - start_date).days

# Для каждой даты устанавливаем свой веc на основании удаленности по времени
unique_dates = df_dates.unique()
unique_dates['weight'] = unique_dates.apply(lambda x: (pd.to_datetime(x[0]) - start_date).days/day_delta, axis=1)

# Join весов к основному датасету
orders['created_at'] = df_dates
orders = orders.join(unique_dates.set_index('created_at'), how='left', on='created_at')

# Нормируем веса
weight_sum = orders.weight.sum()
orders['prob'] = orders.weight/weight_sum
orders_weighted = orders[['id', 'prob']].copy()

# Выбираем n_samples случайных сэмплов в соответствии с весами
n_samples = 2000000
selected_order_ids = np.random.choice(orders_weighted.id.values, n_samples, p=orders_weighted.prob.values, replace=False)


Для каждого клиента обучаемся на последней корзине. Для обучения формируем записи двух видов - положительные и отрицательные. Из корзины клиента случайным образом извлекаем товар и помещаем в положительный таргет. Из оставшихся товаров в ассортименте достаем случайный товар и помещаем его в отрицательный таргет. Количество товаров в истории не учитываем, учитываем только их наличие. Для каждого товара создается два бинарных признака: был ли товар в истории и находится ли он в текущей корзине.

In [ ]:
# Последнюю транзакцию клиента будем использовать как текущую корзину
client_basket = df.groupby('client_idx').apply(lambda x: x[x.transaction_datetime == x.transaction_datetime.max()])
client_history = df.groupby('client_idx').apply(lambda x: x[x.transaction_datetime != x.transaction_datetime.max()])

client_basket.reset_index(drop=True, inplace=True)
client_history.reset_index(drop=True, inplace=True)

products_list = list(product_id_to_idx.values())

def get_negative_sample(products_list, basket):
    """Получение негативного таргета для корзины клиента. 
    Негативный таргет - случайный товар из католога за исключением товаров в корзине."""
    
    neg_samples = list(set(products_list)-set(basket))  
    return np.random.choice(neg_samples, 1)[0]

# Создание отрицательного таргета для каждого клиента
neg_target = client_basket.groupby('client_idx').product_idx\
    .apply(lambda x: get_negative_sample(products_list, x.values))

# Выберем случайный товар для положительного таргета из корзины
target = client_basket.groupby('client_idx').product_idx.apply(lambda x: x.sample(n=1).values[0])
target = target.reset_index()

# Убираем таргет из корзины
without_target = pd.concat([client_basket[['client_idx', 'product_idx']], target, target]).drop_duplicates(keep=False)

# Не учитываем количество товаров в истории
client_history = client_history[['client_idx', 'product_idx']].drop_duplicates() 

# Делаем матричное представление для пар клиент-товар_в_истории
client_history['value'] = [True]*client_history.shape[0]
client_history = client_history.pivot(index='client_idx', columns='product_idx')['value'].fillna(False).add_prefix('h_')

# Делаем матричное представление для пар клиент-товар_в_корзине
without_target['value'] = [True]*without_target.shape[0]
without_target = without_target.pivot(index='client_idx', columns='product_idx')['value'].fillna(False).add_prefix('b_')


Далее добавляем атрибуты клиентов в основной датасет.

In [ ]:
client_full_df['age'] = clients['age']
client_full_df['gender'] = clients['gender']

client_full_df.gender.fillna('U', inplace=True)
client_full_df.age.fillna(mode, inplace=True)

#### Расчет RFM (recency, frequency, monetary) агрегатов:
Для учета персональной истории покупок рассчитываем и добавляем в качестве придикторов RFM-агрегаты. 
Рассчитываемые предикторы показывают
1. Количество дней после покупки товара
2. Количество покупок товара в истории
3. Относительные траты на товар
4. Сумму трат на товар за год


In [ ]:
h_columns = ['last_purch_days_' + str(x) for x in range(len(products_list))]
time_last_purch = np.full(len(products_list), np.inf)

# Количество дней после покупки товара
def get_last_purch_days(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)]

    hist = hist[['product_idx', 'transaction_datetime']]
    hist.drop_duplicates(keep='last', inplace=True)
    hist['cur_tdt'] = x.transaction_datetime
    hist.set_index('product_idx', inplace=True)
    hist['time_last_purch'] = (hist['cur_tdt'] - hist['transaction_datetime']).apply(lambda x: x.days)
    time_last_purch_ = time_last_purch.copy()
    time_last_purch_[hist.index.values] = hist['time_last_purch'].values
   
    return pd.Series(time_last_purch_, index=h_columns)

old_trans_last_purch = old_trans_.apply(get_last_purch_days, axis=1)



h_columns = ['h_' + str(x) for x in range(len(products_list))]
zeros_hist = np.zeros(len(products_list)).astype(int)

# Количество покупок товара в истории
def get_history_products(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)]

    vc = hist.product_idx.value_counts()

    exp_hist = zeros_hist.copy()
    exp_hist[vc.index.values] = vc.values

    return pd.Series(exp_hist, index=h_columns)

old_trans_frequency = old_trans_.apply(get_history_products, axis=1)
old_trans_frequency.columns = ['hist_prod_freq_' + str(x) for x in range(len(products_list))]
old_trans_frequency = old_trans_frequency.div(old_trans_frequency.sum(axis=1), axis=0)



h_columns = ['hist_purch_sum_' + str(x) for x in range(len(products_list))]
zeros_sum = np.zeros(len(products_list))

# Относительные траты на товар
def get_purch_sum(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)]

    hist = hist[['product_idx', 'trn_sum_from_iss']]
    hist = hist.groupby('product_idx').sum()
    
    zeros_sum_ = zeros_sum.copy()
    zeros_sum_[hist.index.values] = hist['trn_sum_from_iss'].values
   
    return pd.Series(zeros_sum_, index=h_columns)

old_trans_purch_summ = old_trans_.apply(get_purch_sum, axis=1)
old_trans_purch_relative_summ = old_trans_purch_summ.div(old_trans_purch_summ.sum(axis=1), axis=0)



h_columns = ['hist_purch_period_sum_' + str(x) for x in range(len(products_list))]
zeros_sum = np.zeros(len(products_list))

# Сумма трат на товар за год
def get_purch_period_sum(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)].copy()
    
    hist['diff_days'] = hist.apply(lambda y: (x.transaction_datetime - y.transaction_datetime).days, axis=1)
    hist = hist[hist['diff_days'] < 366]

    hist = hist[['product_idx', 'trn_sum_from_iss']]
    hist = hist.groupby('product_idx').sum()
    
    zeros_sum_ = zeros_sum.copy()
    zeros_sum_[hist.index.values] = hist['trn_sum_from_iss'].values
   
    return pd.Series(zeros_sum_, index=h_columns)

old_trans_purch_period_summ = old_trans_.apply(get_purch_period_sum, axis=1)

### Обучение
Алгоритм градиентного бустинга обучается на следующем наборе предикторов:

- Вектор наличия товара в истории (длина вектора - кол-во всех товаров в ассортименте)
- Вектор наличия товара в корзине
- Временные предикторы
- Атрибуты клиента (пол, возраст)
- Атрибуты товара в таргете
- Скор от SVD
- RFM агрегаты

Скор от SVD рассчитывается как косинусная близость веркторного представления товара в таргете и суммы представлений товаров в корзине.


In [ ]:
# Для SVD используем все транзакции клиентов, которые (клиенты) участвуют в обучении XGB
train_svd = train_transactions[train_transactions.client_idx.isin(train.index.values)]

# Формирование разряженной матрицы товар-чек
rows = train_svd.transaction_idx.astype(int)
cols = train_svd.product_idx.astype(int)
sp_train = scipy.sparse.coo_matrix((np.ones_like(rows), (rows, cols)))
sp_train = sp_train.toarray() / sp_train.sum(axis=0)
svd_model = RecommenderSVDKNN(emb_dim=500)
svd_model.fit(sp_train.T)

# Подсчет скора от SVD
train['knn_score'] = train.apply(lambda x: cosine_similarity(svd_model.X_train_svd[x.target_product].reshape(1, -1), svd_model.X_train_svd[x[bask].values.astype(bool)].sum(0).reshape(1, -1))[0][0], axis=1)

# Перемешиваем выборку для обучения
train = train.sample(frac=1)

print("start train xgb")

xgb_model = xgb.XGBRegressor(objective="reg:logistic", 
                             #tree_method='gpu_hist', 
                             #gpu_id=0, 
                             random_state=42, 
                             n_jobs=-1, 
                             max_depth=6, 
                             n_estimators=300)

xgb_model.fit(train.drop('target', axis=1).values, train.target.values)

### Валидация

In [ ]:
def gpu_validate(test_slice):
    """Функция валидации тестового датасета. Расчет косинусной близости реализован на pytorch.
    Итерация происходит по каждому клиенту в тестовом датасете. 
    В ходе итерации собирается батч вида клиент-товар, где атрибуты клиента фиксированны, 
    а изменяется только товар в таргете и его атрибуты."""
    
    target_input = test_slice['target_product'].values
    test_input = []

    for index, user in test_slice.drop(products.columns, axis=1).iterrows():

        # Текущая корзина клиента
        basket = user['b_0':'b_199']

        # Собираем батч клиент-товар
        user_df = pd.concat([user] * PRODUCTS_COUNT, axis=1).transpose().drop('target', axis=1)
        user_df = pd.concat([user_df.reset_index(drop=True), products], axis=1)
        
        # Считаем скор SVD на gpu, если доступна видеокарта
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        a = torch.from_numpy(svd_model.X_train_svd).to(device)
        b = torch.from_numpy(svd_model.X_train_svd[basket.values.astype(bool)].sum(0).reshape(1, -1)).to(device)
        a_norm = a / a.norm(dim=1)[:, None]
        b_norm = b / b.norm(dim=1)[:, None]
        res = torch.mm(a_norm, b_norm.transpose(0,1)).cpu().numpy()

        user_df['knn_score'] = res
        user_df = user_df[dataset_columns]

        y_pred = xgb_model.predict(user_df.values)

        # Ранжируем товары, исключаем товары из корзины
        y_pred[basket.values.astype(bool)] = -np.inf
        preds = np.argpartition(y_pred, -3, axis=0)[-3:]
        test_input.append(list(preds))

    return (target_input, test_input)


# На тесте используем только положительный таргет
test = test[test.target == 1]
# Сохраняем товары из таргета для расчета метрики
target_input = test['target_product'].values
# Массив для прогнозирования
test_input = []
# Сохраняем порядок предикторов для XGB
dataset_columns = train.drop('target', axis=1).columns  

result = gpu_validate(test)

score = mean_precision(result[0], result[1])
print('Mean Precision:', score)
scores.append(score)

### Формирование рекомендаций

In [2]:
class XGBRecommender():
    """Класс-обертка для градиентного бустинга."""
    
    def __init__(self):
        """Инициализация класса. Загрузка состояний моделей и датасетов."""
             
        self.svd_model = self.load_model('svd_model')
        self.xgb_model = self.load_model('xgb_model')
        
        self.dataset = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'rfm_client_hist_context_target_product.csv']), index_col=0)
        self.dataset = self.dataset[self.dataset.target == 1]
        self.dataset.drop(['is_new_client'], axis=1, inplace=True)
        
        self.products = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'products.csv']), index_col=0)
        self.prod_num = self.products.shape[0]
        self.products = self.products.sort_index()
        self.products['target_product'] = range(self.prod_num) 
   
        
    def load_model(self, name):
        
        path = ft.reduce(os.path.join, ['..', 'models','pkl' ,name+'.pkl'])
        with open(path, 'rb') as handle:
            return pickle.load(handle)         
    
    def predict_sample(self, user_id, basket, store_idx, time, slot_num):
        """Функция возвращает рекомендованные к покупке товары и их рейтинг.
        Рекомендации осуществляются на основе id клиента и его текущей корзины. """
        
        basket_header = ['b_' + str(x) for x in range(self.prod_num)]
        history_header = ['h_' + str(x) for x in range(self.prod_num)]

        # Формируем историю покупок клиента
        user = self.dataset.loc[user_id].copy()
        user.loc[history_header] = user.loc[basket_header].values | user.loc[history_header].values
        user.loc['h_'+str(user.target_product)] = True

        # Приводим список товаров в корзине к вектору-индикатору наличия товара в корзине
        basket_ = np.full((self.prod_num), False, dtype=bool)
        basket_[basket] = True
        user.loc[basket_header] = basket_
        user.drop(self.products.columns, inplace=True)
        
        # Собираем контекст
        time = pd.to_datetime(time)
        user['store_idx'] = store_idx
        user['product_count'] = np.sum(basket_)
        user['hour'] = time.hour
        user['dayofweek'] = time.dayofweek
        user['day'] = time.day
        user['month'] = time.month
        user['year'] = time.year
        
        # Готовим датасет вида клиент(контекст)-товары для скоринга товаров
        user_df = pd.concat([user] * self.prod_num, axis=1).transpose().drop('target', axis=1)
        user_df = pd.concat([user_df.reset_index(drop=True), self.products], axis=1)
        
        user_df = user_df[self.dataset.drop('target', axis=1).columns]
        
        # Рассчитываем скор SVD_KNN
        user_df['knn_score'] = cosine_similarity(self.svd_model.X_train_svd, self.svd_model.X_train_svd[basket].sum(0).reshape(1, -1))

        y_pred = self.xgb_model.predict(user_df.values)
        
        # Ранжируем, отбираем топ        
        y_pred[basket] = -np.inf

        recs = np.argpartition(y_pred, -slot_num, axis=0)[-slot_num:]
        recs = list(recs[np.argsort(y_pred[recs])])
        recs.reverse()
        
        return recs
